In [3]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('weather.csv', 'weather'),
    ('drone_mule_ops.csv', 'drone'),
    ('logistics.csv', 'logistics'),
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='MySQL@9635',
    database='armydronelogisticsdb'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'D:\Github\Project Repos\Data Analytics\Operation Suraksha Data-Driven Logistics Optimization for the Indian Army'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()


<>:22: SyntaxWarning: invalid escape sequence '\G'
<>:22: SyntaxWarning: invalid escape sequence '\G'
C:\Users\USER\AppData\Local\Temp\ipykernel_10552\2548819970.py:22: SyntaxWarning: invalid escape sequence '\G'
  folder_path = 'D:\Github\Project Repos\Data Analytics\Operation Suraksha Data-Driven Logistics Optimization for the Indian Army'


Processing weather.csv
NaN values before replacement:
Date               0
Location           0
Temperature_C      0
Snowfall_cm        0
Visibility_m       0
Wind_Speed_kmph    0
dtype: int64

Processing drone_mule_ops.csv
NaN values before replacement:
Delivery_ID       0
Mode              0
Battery_Used_%    0
Payload_kg        0
Distance_km       0
Avg_Speed_kmph    0
Weather_Impact    0
dtype: int64

Processing logistics.csv
NaN values before replacement:
Mission_ID            0
Source_Base           0
Destination_Post      0
Transport_Mode        0
Departure_Time        0
Arrival_Time          0
Status                0
Delay_Reason         53
Critical_Supplies     0
dtype: int64

